# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [7]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
from patsy import dmatrix
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
df = df.dropna()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [21]:
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [16]:
y, x = patsy.dmatrices('renda ~ sexo + posse_de_imovel + posse_de_veiculo + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', df[:5])
x

DesignMatrix with shape (5, 13)
  Columns:
    ['Intercept',
     'sexo[T.M]',
     'posse_de_imovel[T.True]',
     'posse_de_veiculo[T.True]',
     'tipo_renda[T.Empresário]',
     'tipo_renda[T.Servidor público]',
     'educacao[T.Superior completo]',
     'estado_civil[T.Solteiro]',
     'tipo_residencia[T.Governamental]',
     'qtd_filhos',
     'idade',
     'tempo_emprego',
     'qt_pessoas_residencia']
  Terms:
    'Intercept' (column 0)
    'sexo' (column 1)
    'posse_de_imovel' (column 2)
    'posse_de_veiculo' (column 3)
    'tipo_renda' (columns 4:6)
    'educacao' (column 6)
    'estado_civil' (column 7)
    'tipo_residencia' (column 8)
    'qtd_filhos' (column 9)
    'idade' (column 10)
    'tempo_emprego' (column 11)
    'qt_pessoas_residencia' (column 12)
  (to view full data, use np.asarray(this_obj))

In [11]:
y

DesignMatrix with shape (5, 1)
    renda
  8060.34
  1852.15
  2253.89
  6600.77
  6475.97
  Terms:
    'renda' (column 0)

In [27]:
reg = smf.ols('renda ~ sexo + posse_de_imovel + posse_de_veiculo + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     178.0
Date:                Fri, 14 Apr 2023   Prob (F-statistic):               0.00
Time:                        09:44:45   Log-Likelihood:            -1.2879e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12402   BIC:                         2.578e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                        -5626.6247   2500.235     -2.250      0.024   -1.05e+04    -725.776
C(sexo)[T.M]                      6052.8592    155.859     38.836      0.000    5747.352    6358.367
posse_de_imovel[T.True]            366.5747    148.723      2.465      0.014      75.054     658.095
posse_de_veiculo[T.True]            48.9098    150.374      0.325      0.745    -245.846     343.666
tipo_renda[T.Bolsista]           -1357.7175   2564.833     -0.529      0.597   -6385.188    3669.753
tipo_renda[T.Empresário]           823.1460    158.826      5.183      0.000     511.822    1134.470
tipo_renda[T.Pensionista]        -2682.6331   2565.489     -1.046      0.296   -7711.391    2346.125
tipo_renda[T.Servidor público]     137.0942    236.327      0.580      0.562    -326.143     600.332
educacao[T.Pós graduação]         1090.5064   1690.396      0.645      0.519   -2222.932    4403.945
educacao[T.Secundário]             607.3280    765.686      0.793      0.428    -893.536    2108.192
educacao[T.Superior completo]     1225.5587    768.564      1.595      0.111    -280.947    2732.064
educacao[T.Superior incompleto]    220.4722    827.097      0.267      0.790   -1400.767    1841.712
estado_civil[T.Separado]          1161.1096   1182.969      0.982      0.326   -1157.693    3479.912
estado_civil[T.Solteiro]           819.9016   1158.206      0.708      0.479   -1450.362    3090.165
estado_civil[T.União]             -475.3334    267.717     -1.776      0.076   -1000.101      49.434
estado_civil[T.Viúvo]             1053.5626   1229.396      0.857      0.391   -1356.245    3463.370
tipo_residencia[T.Casa]           -298.3088    575.821     -0.518      0.604   -1427.008     830.390
tipo_residencia[T.Com os pais]    -388.7990    641.844     -0.606      0.545   -1646.914     869.316
tipo_residencia[T.Comunitário]    -459.8284   1120.521     -0.410      0.682   -2656.224    1736.567
tipo_residencia[T.Estúdio]          77.2381   1056.186      0.073      0.942   -1993.051    2147.527
tipo_residencia[T.Governamental]    73.8042    701.702      0.105      0.916   -1301.641    1449.249
qtd_filhos                       -1126.6107   1147.916     -0.981      0.326   -3376.703    1123.482
idade                               42.0025      8.517      4.931      0.000      25.307      58.698
tempo_emprego                      554.7551     11.044     50.230      0.000     533.107     576.404
qt_pessoas_residencia             1269.8493   1145.868      1.108      0.268    -976.229    3515.928
==============================================================================
Omnibus:                    17568.773   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9884998.164
Skew:             

In [32]:
df[['sexo', 'posse_de_imovel', 'posse_de_veiculo', 'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia']].value_counts()

sexo  posse_de_imovel  posse_de_veiculo  qtd_filhos  tipo_renda        educacao             estado_civil  tipo_residencia  idade  tempo_emprego  qt_pessoas_residencia
M     True             True              0           Servidor público  Superior completo    Solteiro      Casa             27     2.263014       1.0                      20
                                         2           Assalariado       Superior completo    Casado        Casa             36     6.786301       4.0                      20
                       False             1           Assalariado       Superior incompleto  Casado        Casa             37     15.449315      3.0                      18
F     True             True              0           Assalariado       Secundário           Casado        Casa             46     19.901370      2.0                      18
                       False             0           Assalariado       Secundário           Casado        Casa             37     15.035616  

In [36]:
df['sexo'].value_counts()

F    7901
M    4526
Name: sexo, dtype: int64

In [37]:
df['posse_de_imovel'].value_counts()

True     8116
False    4311
Name: posse_de_imovel, dtype: int64

In [38]:
df['posse_de_veiculo'].value_counts()

False    7160
True     5267
Name: posse_de_veiculo, dtype: int64

In [39]:
df['qtd_filhos'].value_counts()

0     7887
1     2973
2     1358
3      183
4       17
7        5
14       2
5        2
Name: qtd_filhos, dtype: int64

In [40]:
df['tipo_renda'].value_counts()

Assalariado         7633
Empresário          3508
Servidor público    1268
Bolsista               9
Pensionista            9
Name: tipo_renda, dtype: int64

In [41]:
df['educacao'].value_counts()

Secundário             7045
Superior completo      4695
Superior incompleto     558
Primário                103
Pós graduação            26
Name: educacao, dtype: int64

In [42]:
df['estado_civil'].value_counts()

Casado      8897
Solteiro    1543
União        924
Separado     739
Viúvo        324
Name: estado_civil, dtype: int64

In [43]:
df['tipo_residencia'].value_counts()

Casa             11071
Com os pais        674
Governamental      360
Aluguel            183
Estúdio             75
Comunitário         64
Name: tipo_residencia, dtype: int64

In [44]:
df['idade'].value_counts()

40    533
37    466
43    458
32    455
33    441
39    438
38    433
27    432
30    429
41    418
46    415
28    409
34    407
31    403
44    398
42    392
36    382
47    379
29    377
35    372
45    360
50    358
48    343
51    316
54    311
52    303
49    302
53    291
26    176
56    171
55    148
60    146
25    133
57    107
59    104
58    103
24     99
62     44
61     41
63     39
64     34
23     26
22     15
65      8
66      8
67      4
Name: idade, dtype: int64

In [45]:
df['tempo_emprego'].value_counts()

4.216438     38
6.934247     30
7.520548     29
15.449315    27
5.717808     26
             ..
31.202740     1
5.279452      1
0.652055      1
2.827397      1
2.726027      1
Name: tempo_emprego, Length: 2589, dtype: int64

In [46]:
df['qt_pessoas_residencia'].value_counts()

2.0     6469
3.0     2486
1.0     1975
4.0     1292
5.0      179
6.0       18
9.0        5
15.0       2
7.0        1
Name: qt_pessoas_residencia, dtype: int64

In [51]:
reg = smf.ols('renda ~ sexo + posse_de_imovel + posse_de_veiculo + C(qtd_filhos, Treatment(0)) + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + C(qt_pessoas_residencia, Treatment(2))', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.257
Model:                            OLS   Adj. R-squared:                  0.255
Method:                 Least Squares   F-statistic:                     126.0
Date:                Fri, 14 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:07:19   Log-Likelihood:            -1.2878e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12392   BIC:                         2.579e+05
Df Model:                          34                                         
Covariance Type:            nonrobust                                         
==================================================================================================================
                                                     coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------
Intercept                                      -3080.3185   1009.316     -3.052      0.002   -5058.735   -1101.902
sexo[T.M]                                       6049.1839    156.134     38.743      0.000    5743.137    6355.231
posse_de_imovel[T.True]                          382.5304    148.998      2.567      0.010      90.471     674.590
posse_de_veiculo[T.True]                          49.5337    150.514      0.329      0.742    -245.497     344.564
C(qtd_filhos, Treatment(0))[T.1]               -1256.9014   1188.092     -1.058      0.290   -3585.746    1071.944
C(qtd_filhos, Treatment(0))[T.2]               -1786.6621   2367.769     -0.755      0.451   -6427.858    2854.534
C(qtd_filhos, Treatment(0))[T.3]               -3925.0136   5171.182     -0.759      0.448   -1.41e+04    6211.307
C(qtd_filhos, Treatment(0))[T.4]                 -94.7998   4558.208     -0.021      0.983   -9029.596    8839.996
C(qtd_filhos, Treatment(0))[T.5]                 110.9318   4363.566      0.025      0.980   -8442.336    8664.199
C(qtd_filhos, Treatment(0))[T.7]                 402.1400   1718.668      0.234      0.815   -2966.717    3770.997
C(qtd_filhos, Treatment(0))[T.14]              -3364.6001   2779.246     -1.211      0.226   -8812.354    2083.154
tipo_renda[T.Bolsista]                         -1368.1345   2564.862     -0.533      0.594   -6395.663    3659.394
tipo_renda[T.Empresário]                         827.9333    158.868      5.211      0.000     516.528    1139.339
tipo_renda[T.Pensionista]                      -2833.6267   2582.972     -1.097      0.273   -7896.654    2229.400
tipo_renda[T.Servidor público]                   126.5053    236.489      0.535      0.593    -337.049     590.060
educacao[T.Pós graduação]                       1200.7567   1693.607      0.709      0.478   -2118.976    4520.490
educacao[T.Secundário]                           636.9745    765.807      0.832      0.406    -864.127    2138.076
educacao[T.Superior completo]                   1245.6779    768.653      1.621      0.105    -261.001    2752.357
educacao[T.Superior incompleto]                  279.9958    827.403      0.338      0.735   -1341.843    1901.834
estado_civil[T.Separado]                        1277.8366   1193.083      1.071      0.284   -1060.792    3616.465
estado_civil[T.Solteiro]                         926.2892   1172.370      0.790      0.429   -1371.738    3224.316
estado_civil[T.União]                           -450.7774    267.932     -1.682      0.093    -975.966      74.411
estado_civil[T.Viúvo]                           1174.8616   1246.235      0.943      0.346   -1267.953    3617.677
tipo_residencia[T.Casa]                         -309.6375    576.050     -0.538      0.591   

In [52]:
reg = smf.ols('renda ~ sexo + posse_de_imovel + tipo_renda + idade + tempo_emprego ', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.254
Model:                            OLS   Adj. R-squared:                  0.254
Method:                 Least Squares   F-statistic:                     528.6
Date:                Fri, 14 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:11:19   Log-Likelihood:            -1.2881e+05
No. Observations:               12427   AIC:                         2.576e+05
Df Residuals:                   12418   BIC:                         2.577e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                      -2481.0546    336.397     -7.375      0.000   -3140.445   -1821.664
sexo[T.M]                       6091.0534    145.795     41.778      0.000    5805.273    6376.834
posse_de_imovel[T.True]          386.3605    145.543      2.655      0.008     101.074     671.647
tipo_renda[T.Bolsista]          -893.9027   2563.590     -0.349      0.727   -5918.936    4131.131
tipo_renda[T.Empresário]         853.0427    157.504      5.416      0.000     544.311    1161.775
tipo_renda[T.Pensionista]      -2579.2216   2562.352     -1.007      0.314   -7601.828    2443.385
tipo_renda[T.Servidor público]   249.8860    234.926      1.064      0.287    -210.606     710.378
idade                             38.7817      7.838      4.948      0.000      23.419      54.145
tempo_emprego                    554.9459     11.018     50.367      0.000     533.349     576.543
==============================================================================
Omnibus:                    17545.490   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9801698.147
Skew:                           8.106   Prob(JB):                         0.00
Kurtosis:                     139.627   Cond. No.                     1.58e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [53]:
reg = smf.ols('renda ~ sexo + posse_de_imovel + idade + tempo_emprego ', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  renda   R-squared:                       0.252
Model:                            OLS   Adj. R-squared:                  0.252
Method:                 Least Squares   F-statistic:                     1047.
Date:                Fri, 14 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:12:08   Log-Likelihood:            -1.2882e+05
No. Observations:               12427   AIC:                         2.577e+05
Df Residuals:                   12422   BIC:                         2.577e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept               -2121.6370    329.140     -6.446      0.000   -2766.803   -1476.471
sexo[T.M]                6045.7894    145.591     41.526      0.000    5760.409    6331.169
posse_de_imovel[T.True]   394.7377    145.657      2.710      0.007     109.228     680.247
idade                      37.5533      7.838      4.791      0.000      22.189      52.918
tempo_emprego             550.4590     10.903     50.485      0.000     529.087     571.831
==============================================================================
Omnibus:                    17506.453   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9670624.952
Skew:                           8.073   Prob(JB):                         0.00
Kurtosis:                     138.705   Cond. No.                         206.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
reg = smf.ols('np.log(renda) ~ sexo + posse_de_imovel + np.log(idade) + np.log(tempo_emprego) ', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.309
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     1390.
Date:                Fri, 14 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:18:16   Log-Likelihood:                -14018.
No. Observations:               12427   AIC:                         2.805e+04
Df Residuals:                   12422   BIC:                         2.808e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   6.0089      0.108     55.483      0.000       5.797       6.221
sexo[T.M]                   0.7674      0.014     54.255      0.000       0.740       0.795
posse_de_imovel[T.True]     0.0790      0.014      5.579      0.000       0.051       0.107
np.log(idade)               0.3652      0.030     12.248      0.000       0.307       0.424
np.log(tempo_emprego)       0.3562      0.007     51.523      0.000       0.343       0.370
==============================================================================
Omnibus:                       11.861   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               11.845
Skew:                           0.074   Prob(JB):                      0.00268
Kurtosis:                       3.031   Cond. No.                         71.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
reg = smf.ols('np.log(renda) ~ sexo + posse_de_imovel + idade + np.log(tempo_emprego) ', data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.310
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     1392.
Date:                Fri, 14 Apr 2023   Prob (F-statistic):               0.00
Time:                        10:18:19   Log-Likelihood:                -14015.
No. Observations:               12427   AIC:                         2.804e+04
Df Residuals:                   12422   BIC:                         2.808e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   6.9747      0.032    215.264      0.000       6.911       7.038
sexo[T.M]                   0.7669      0.014     54.270      0.000       0.739       0.795
posse_de_imovel[T.True]     0.0788      0.014      5.563      0.000       0.051       0.107
idade                       0.0093      0.001     12.491      0.000       0.008       0.011
np.log(tempo_emprego)       0.3565      0.007     51.678      0.000       0.343       0.370
==============================================================================
Omnibus:                       11.504   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               11.487
Skew:                           0.073   Prob(JB):                      0.00320
Kurtosis:                       3.030   Cond. No.                         205.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Adotando as premissas indicadas na aula, o último modelo apresentou uma performace melhor que os anteriores. Ao aplicar a função logarítmica nas variáveis 'Renda e Tempo de Emprego' a regressão apresentou dados mais consistentes, melhorando o R², o R² ajustado e o AIC. 